In [1]:
import argparse

import cv2
import numpy as np
import torch

from models.with_mobilenet import PoseEstimationWithMobileNet
from modules.keypoints import extract_keypoints, group_keypoints
from modules.load_state import load_state
from modules.pose import Pose, track_poses
from val import normalize, pad_width

from modules.one_euro_filter import OneEuroFilter
from modules.keypoints import BODY_PARTS_KPT_IDS, BODY_PARTS_PAF_IDS

import matplotlib as plt
from matplotlib import cm
from modules.pose import Pose

from demo import *

In [2]:
import urllib
urllib.request.urlretrieve ("https://download.01.org/opencv/openvino_training_extensions/models/human_pose_estimation/checkpoint_iter_370000.pth",
                    "model/pretrained.pth")

('model/pretrained.pth', <http.client.HTTPMessage at 0x7f9da0824668>)

The pose may contain up to 18 keypoints: ears, eyes, nose, neck, shoulders, elbows, wrists, hips, knees, and ankles.

For  multi-person  pose  es-timation,  most  approaches have  used  a  top-down  strategy  that  firstdetects  people  and  then  have  estimated  the  pose  of  eachperson  independently  on  each  detected  region

<img src="images/openpose_pipeline.png">

<img src="images/openpose-body-architecture-1024x291.png">

In [3]:
img = cv2.imread('images/joker.jpeg', cv2.IMREAD_COLOR)
img_disp = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

In [7]:
net = PoseEstimationWithMobileNet()
checkpoint = torch.load('model/pretrained.pth', map_location='cpu')
load_state(net, checkpoint)
net = net.eval()



In [ ]:
plt.imshow(img_disp)

In [ ]:
stride = 8
upsample_ratio = 8
num_keypoints = Pose.num_kpts
orig_img = img.copy()

heatmaps, pafs, scale, pad = infer_fast(net, img, 512, stride, upsample_ratio, cpu=True)

In [ ]:
plt.figure(figsize=[15, 10])
plt.imshow(np.max(heatmaps[:,:,:18], axis=2))
plt.imshow(cv2.resize(img_disp, (776,512)), alpha=0.3)

In [ ]:
plt.figure(figsize=[15, 10])
plt.imshow(1 - pafs[:,:,2])
plt.imshow(cv2.resize(img_disp, (776,512)), alpha=0.3)

In [ ]:
plt.figure(figsize=[15, 10])
plt.imshow(np.abs(pafs[:,:,14]))
plt.imshow(cv2.resize(img_disp, (776,512)), alpha=0.3)

In [ ]:
skip=(slice(None,None,2),slice(None,None,2))
plt.figure(figsize=[15, 10])
plt.quiver(pafs[160:220, 320:420, 14][skip], - pafs[160:220, 320:420, 15][skip], color='cyan',scale=30)
plt.imshow(cv2.resize(img_disp, (776,512))[160:220, 320:420], extent=(-0.5, 50-0.5, 30-0.5, -0.5))

<img src="images/keypoint_extraction.png">

In [ ]:
total_keypoints_num = 0
all_keypoints_by_type = []
for kpt_idx in range(num_keypoints):  # 19th for bg
    total_keypoints_num += extract_keypoints(heatmaps[:, :, kpt_idx], all_keypoints_by_type, total_keypoints_num)

<img src="images/keypoints.png">

In [ ]:
keypoints = {
        0: "nose",
        1: "neck",
        2: "left_shoulder",
        3: "left_elbow",
        4: "left_wrist",
        5: "right_shoulder",
        6: "right_elbow",
        7: "right_wrist",
        8: "left_hip",
        9: "left_knee",
        10: "left_ankle",
        11: "right_hip",
        12: "right_knee",
        13: "right_ankle",
        14: "left_eye",
        15: "right_eye",
        16: "left_ear",
        17: "right_ear",
    }

In [ ]:
[f'{len(x)} keypoints extracted for {keypoints[i]}' for i, x in enumerate(all_keypoints_by_type)]

Too many point detected for ankles

In [ ]:
img_with_ankle_keypoints = cv2.resize(img_disp.copy(), (776,512))
for ankle_keypoint in all_keypoints_by_type[10]:
    img_with_ankle_keypoints = cv2.circle(img_with_ankle_keypoints,
                                          ankle_keypoint[:2], 10, color=(0, 255, 255), thickness=-1)
    
plt.imshow(img_with_ankle_keypoints)

In [ ]:
def get_positions(all_keypoints_by_type, pafs):
    pose_entries, all_keypoints = group_keypoints(all_keypoints_by_type, pafs, demo=True)
    for kpt_id in range(all_keypoints.shape[0]):
        all_keypoints[kpt_id, 0] = (all_keypoints[kpt_id, 0] * stride / upsample_ratio - pad[1]) / scale
        all_keypoints[kpt_id, 1] = (all_keypoints[kpt_id, 1] * stride / upsample_ratio - pad[0]) / scale
    current_poses = []
    for n in range(len(pose_entries)):
        if len(pose_entries[n]) == 0:
            continue
        pose_keypoints = np.ones((num_keypoints, 2), dtype=np.int32) * -1
        for kpt_id in range(num_keypoints):
            if pose_entries[n][kpt_id] != -1.0:  # keypoint was found
                pose_keypoints[kpt_id, 0] = int(all_keypoints[int(pose_entries[n][kpt_id]), 0])
                pose_keypoints[kpt_id, 1] = int(all_keypoints[int(pose_entries[n][kpt_id]), 1])
        pose = Pose(pose_keypoints, pose_entries[n][18])
        current_poses.append(pose)
        
    return current_poses

In [ ]:
all_positions = get_positions(all_keypoints_by_type, pafs)
colormap = cm.tab10.colors

for i, pose in enumerate(all_positions):
    pose.draw(img_disp, np.array(colormap[i % len(colormap)])*255)

In [ ]:
plt.figure(figsize=[10, 10])
plt.imshow(img_disp)